In [ ]:
!pip install -q -U \
  "bitsandbytes==0.46.0" \
  "transformers==4.41.2" \
  "peft==0.11.1" \
  "accelerate==0.31.0" \
  "datasets==2.19.2" \
  "trl==0.8.6" \
  "huggingface_hub" \
  "minijinja" \
  "triton==3.2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer
from huggingface_hub import login
import os

In [ ]:
import random
import numpy as np
import torch

def fix_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
from huggingface_hub import login
import os

# Recomenda-se armazenar o token em uma variável de ambiente ou prompt para segurança
# ou diretamente como string (menos recomendado para repositórios públicos)
# login(token="hf_SEU_TOKEN_AQUI")
# OU
# token = os.environ.get("HF_TOKEN") # Se você definiu a variável de ambiente HF_TOKEN
# login(token=token)
# OU
login() # Será solicitado que você insira seu token no console do Jupyter

In [ ]:
MODEL_ID_FINETUNE = "meta-llama/Meta-Llama-3-8B-Instruct"

NEW_MODEL_NAME = f"{MODEL_ID_FINETUNE.split('/')[-1]}-synthetic-qlora"
OUTPUT_DIR = f"./{NEW_MODEL_NAME}-results"

print(f"Modelo base para fine-tuning: {MODEL_ID_FINETUNE}")
print(f"Nome do novo adaptador: {NEW_MODEL_NAME}")

Modelo base para fine-tuning: meta-llama/Meta-Llama-3-8B-Instruct
Nome do novo adaptador: Meta-Llama-3-8B-Instruct-synthetic-qlora


In [ ]:
import torch

# Configuração de Quantização
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",       # Tipo de quantização em NormalFloat 4
    bnb_4bit_compute_dtype=torch.bfloat16, # Dtype para cômputo (float16, bfloat16)
    bnb_4bit_use_double_quant=False, # Usar quantização dupla (economiza um pouco mais de memória)
)

In [ ]:
print(f"Carregando modelo base: {MODEL_ID_FINETUNE}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID_FINETUNE,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
print("Modelo carregado.")


tokenizer = AutoTokenizer.from_pretrained(MODEL_ID_FINETUNE, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Tokenizer carregado.")
print(f"PAD token ID: {tokenizer.pad_token_id}, EOS token ID: {tokenizer.eos_token_id}")

Carregando modelo base: meta-llama/Meta-Llama-3-8B-Instruct


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Tokenizer carregado.
PAD token ID: 128009, EOS token ID: 128009


In [ ]:
from datasets import load_dataset

TRAIN_DATA_PATH = "train_formatted.jsonl"

print(f"Carregando dataset de treino de: '{TRAIN_DATA_PATH}'")
try:
    train_dataset = load_dataset("json", data_files=TRAIN_DATA_PATH, split="train")

    print("\nDataset de treino carregado com sucesso!")
    print(f"Total de exemplos de treino: {len(train_dataset)}")

    print("\n--- Exemplo de uma amostra carregada ---")
    print(train_dataset[0]['text'])
    print("------------------------------------------")

except Exception as e:
    print(f"\n Erro ao carregar o arquivo. Verifique se o caminho '{TRAIN_DATA_PATH}' está correto.")
    print(f"   Erro: {e}")

Carregando dataset de treino de: 'train_formatted.jsonl'


Generating train split: 0 examples [00:00, ? examples/s]


Dataset de treino carregado com sucesso!
Total de exemplos de treino: 7000

--- Exemplo de uma amostra carregada ---
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

   You are an expert model specializing in the task of converting natural language text into SQL queries. Your role is to answer user questions by generating syntactically correct SQL queries based on the provided database schema. Your response must be solely the SQL query corresponding to the natural language text. Generate the SQL query in a single line. Do not insert line breaks within the generated query. Use ONLY the tables and columns provided in the ‘Database Schema’ section, referring to them correctly. Do not make up column or table names. Use table aliases (e.g., T1, T2) only when a JOIN operation between multiple tables is required, either to disambiguate columns with the same name or to improve clarity. For queries involving a single table, do not use aliases. Refer to the examples below to underst

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "o_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    max_steps=50,  # Força o treinamento a parar após 50 passos (~30 minutos)
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    save_steps=25, # Salvar na metade e no final do treino
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    bf16=True if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none",
    evaluation_strategy="no",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,             # <-- CORREÇÃO: Usando a variável correta 'train_dataset'
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=512,                      # Ponto de partida seguro. Aumentar se os schemas forem muito longos.
    args=training_arguments,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# Para limpar a memória da GPU antes de treinar (útil no Colab)e
torch.cuda.empty_cache()

# Iniciar o treinamento
print("Iniciando o treinamento QLoRA...")
try:
    trainer.train()
    print("Treinamento concluído!")
except Exception as e:
    print(f"Ocorreu um erro durante o treinamento: {e}")
    print("Isso pode ser devido a limitações de memória da GPU no Colab.")
    print("Tente reduzir 'per_device_train_batch_size', 'max_seq_length', ou 'r' na LoraConfig.")
    print("Verifique também se o otimizador 'paged_adamw_8bit' está funcionando ou use 'adamw_torch'.")
    # Se o treinamento falhar, ainda podemos tentar salvar um estado intermediário se houver checkpoints
    # ou simplesmente parar.

Iniciando o treinamento QLoRA...


Step,Training Loss
10,1.283900
20,0.196600
30,0.166400
40,0.143200
50,0.144100


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Treinamento concluído!


In [ ]:
import os
import torch # Certifique-se de que torch está importado

def save_adapter(trainer_instance, tokenizer_instance, base_output_dir, adapter_folder_name):
    """
    Salva o adaptador LoRA treinado e o tokenizer em um subdiretório específico.

    Args:
        trainer_instance (Trainer): A instância do Trainer que acabou de ser treinada.
        tokenizer_instance (Tokenizer): O tokenizer usado no treinamento.
        base_output_dir (str): O diretório principal de resultados.
        adapter_folder_name (str): O nome específico para a pasta deste adaptador.
    """
    # 1. Cria o caminho completo para o diretório do novo adaptador
    adapter_path = os.path.join(base_output_dir, adapter_folder_name)

    print(f"\nIniciando o salvamento do adaptador em '{adapter_path}'...")

    try:
        # 2. Salva os pesos do adaptador LoRA e sua configuração
        trainer_instance.save_model(adapter_path)
        print(f"✅ Adaptador LoRA salvo com sucesso em: {adapter_path}")

        # 3. Salva os arquivos do tokenizer no mesmo diretório para garantir consistência
        tokenizer_instance.save_pretrained(adapter_path)
        print(f"✅ Tokenizer salvo com sucesso em: {adapter_path}")

    except Exception as e:
        print(f"❌ Erro ao salvar o modelo/adaptador: {e}")
        print("Isso pode ter ocorrido se o treinamento não foi iniciado ou concluído com sucesso.")


# --- Exemplo de Como Usar a Nova Função ---

# Dê um nome descritivo para o adaptador que você acabou de treinar.
# Incluir os hiperparâmetros no nome é uma ótima prática!
nome_do_meu_experimento = "Llama3-8B_Qlora1"

# Chame a função para salvar tudo de forma organizada
save_adapter(
    trainer_instance=trainer,
    tokenizer_instance=tokenizer,
    base_output_dir=OUTPUT_DIR,
    adapter_folder_name=nome_do_meu_experimento
)


Iniciando o salvamento do adaptador em './Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora1'...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Adaptador LoRA salvo com sucesso em: ./Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora1
✅ Tokenizer salvo com sucesso em: ./Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora1


In [ ]:
import os

# Nome da pasta final que você quer baixar (o nome do seu experimento)
folder_name_to_download = "Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora1"

# Caminho completo para a pasta
full_folder_path = "Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora1"
# Nome que daremos ao arquivo compactado
zip_output_filename = f"{folder_name_to_download}.zip"

print(f"Compactando a pasta '{full_folder_path}' em '{zip_output_filename}'...")

# Comando do sistema para criar o arquivo .zip (-r para recursivo, -q para quieto)
!zip -r -q {zip_output_filename} {full_folder_path}

print("✅ Compactação concluída!")
print(f"Agora você pode encontrar o arquivo '{zip_output_filename}' no painel de arquivos à esquerda, clicar com o botão direito sobre ele e selecionar 'Fazer o download'.")

Compactando a pasta 'Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora1' em 'Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora1.zip'...
✅ Compactação concluída!
Agora você pode encontrar o arquivo 'Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora1.zip' no painel de arquivos à esquerda, clicar com o botão direito sobre ele e selecionar 'Fazer o download'.


In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "o_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    max_steps = 200,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    save_steps= 25 ,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    bf16=True if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none",
    evaluation_strategy="no",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=512,
    args=training_arguments,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# Para limpar a memória da GPU antes de treinar (útil no Colab)e
torch.cuda.empty_cache()

# Iniciar o treinamento
print("Iniciando o treinamento QLoRA...")
try:
    trainer.train()
    print("Treinamento concluído!")
except Exception as e:
    print(f"Ocorreu um erro durante o treinamento: {e}")
    print("Isso pode ser devido a limitações de memória da GPU no Colab.")
    print("Tente reduzir 'per_device_train_batch_size', 'max_seq_length', ou 'r' na LoraConfig.")
    print("Verifique também se o otimizador 'paged_adamw_8bit' está funcionando ou use 'adamw_torch'.")

Iniciando o treinamento QLoRA...


Step,Training Loss
10,1.541300
20,0.242700
30,0.164400
40,0.119500
50,0.105800
60,0.097500
70,0.078400


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Step,Training Loss
10,1.541300
20,0.242700
30,0.164400
40,0.119500
50,0.105800
60,0.097500
70,0.078400
80,0.060100
90,0.049500
100,0.053100


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in ver

Treinamento concluído!


In [ ]:
import os
import torch # Certifique-se de que torch está importado

def save_adapter(trainer_instance, tokenizer_instance, base_output_dir, adapter_folder_name):
    """
    Salva o adaptador LoRA treinado e o tokenizer em um subdiretório específico.

    Args:
        trainer_instance (Trainer): A instância do Trainer que acabou de ser treinada.
        tokenizer_instance (Tokenizer): O tokenizer usado no treinamento.
        base_output_dir (str): O diretório principal de resultados.
        adapter_folder_name (str): O nome específico para a pasta deste adaptador.
    """
    # 1. Cria o caminho completo para o diretório do novo adaptador
    adapter_path = os.path.join(base_output_dir, adapter_folder_name)

    print(f"\nIniciando o salvamento do adaptador em '{adapter_path}'...")

    try:
        # 2. Salva os pesos do adaptador LoRA e sua configuração
        trainer_instance.save_model(adapter_path)
        print(f"✅ Adaptador LoRA salvo com sucesso em: {adapter_path}")

        # 3. Salva os arquivos do tokenizer no mesmo diretório para garantir consistência
        tokenizer_instance.save_pretrained(adapter_path)
        print(f"✅ Tokenizer salvo com sucesso em: {adapter_path}")

    except Exception as e:
        print(f"❌ Erro ao salvar o modelo/adaptador: {e}")
        print("Isso pode ter ocorrido se o treinamento não foi iniciado ou concluído com sucesso.")


# --- Exemplo de Como Usar a Nova Função ---

# Dê um nome descritivo para o adaptador que você acabou de treinar.
# Incluir os hiperparâmetros no nome é uma ótima prática!
nome_do_meu_experimento = "Llama3-8B_Qlora2"

# Chame a função para salvar tudo de forma organizada
save_adapter(
    trainer_instance=trainer,
    tokenizer_instance=tokenizer,
    base_output_dir=OUTPUT_DIR,
    adapter_folder_name=nome_do_meu_experimento
)


Iniciando o salvamento do adaptador em './Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora2'...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Adaptador LoRA salvo com sucesso em: ./Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora2
✅ Tokenizer salvo com sucesso em: ./Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora2


In [ ]:
import os

# Nome da pasta final que você quer baixar (o nome do seu experimento)
folder_name_to_download = "Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora1"

# Caminho completo para a pasta
full_folder_path = "Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora2"
# Nome que daremos ao arquivo compactado
zip_output_filename = f"{folder_name_to_download}.zip"

print(f"Compactando a pasta '{full_folder_path}' em '{zip_output_filename}'...")

# Comando do sistema para criar o arquivo .zip (-r para recursivo, -q para quieto)
!zip -r -q {zip_output_filename} {full_folder_path}

print("✅ Compactação concluída!")
print(f"Agora você pode encontrar o arquivo '{zip_output_filename}' no painel de arquivos à esquerda, clicar com o botão direito sobre ele e selecionar 'Fazer o download'.")

Compactando a pasta 'Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora2' em 'Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora1.zip'...
✅ Compactação concluída!
Agora você pode encontrar o arquivo 'Meta-Llama-3-8B-Instruct-synthetic-qlora-results/Llama3-8B_Qlora1.zip' no painel de arquivos à esquerda, clicar com o botão direito sobre ele e selecionar 'Fazer o download'.
